# Project 1
## Quantitative Analysis of New York City Traffic Congestion
### How is the pause of issuing FHV licenses improving traffic congestion and earnings in Manhattan Borough









###### Start the project by importing library and packages

In [2]:
import pandas as pd
import csv
import numpy as np
import seaborn as sns
import datetime
import matplotlib.pyplot as plt
import glob
import folium
import json
import os
from sklearn.impute import SimpleImputer # used for handling missing data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder # used for encoding categorical data
from sklearn.model_selection import train_test_split # used for splitting training and testing data
from sklearn.preprocessing import StandardScaler # used for feature scaling
from folium.plugins import FastMarkerCluster
from folium.plugins import HeatMap
from bokeh.plotting import figure, show
from bokeh.tile_providers import get_provider, Vendors
from bokeh.io import save, reset_output, output_notebook
import geopandas as gpd

###### Filtered warnings

In [3]:
import warnings
warnings.filterwarnings("ignore")

###### Read the dataset and use pickle to save the run time

In [4]:
gre_19_07 = pd.read_csv("green_tripdata_2019-07.csv", header=0,low_memory=False,  encoding='latin-1')
gre_19_08 = pd.read_csv("green_tripdata_2019-08.csv", header=0,low_memory=False,  encoding='latin-1')
yel_19_07 = pd.read_csv("yellow_tripdata_2019-07.csv", header=0,low_memory=False,  encoding='latin-1')
yel_19_08 = pd.read_csv("yellow_tripdata_2019-08.csv", header=0,low_memory=False,  encoding='latin-1')

gre_18_07 = pd.read_csv("green_tripdata_2018-07.csv", header=0,low_memory=False,  encoding='latin-1')
gre_18_08 = pd.read_csv("green_tripdata_2018-08.csv", header=0,low_memory=False,  encoding='latin-1')
yel_18_07 = pd.read_csv("yellow_tripdata_2019-07.csv", header=0,low_memory=False,  encoding='latin-1')
yel_18_08 = pd.read_csv("yellow_tripdata_2019-08.csv", header=0,low_memory=False,  encoding='latin-1')


gre_19_07.to_pickle('green_tripdata_2019_07.pkl')
gre_19_08.to_pickle('green_tripdata_2019_08.pkl')
yel_19_07.to_pickle('yellow_tripdata_2019_07.pkl')
yel_19_08.to_pickle('yellow_tripdata_2019_08.pkl')

gre_18_07.to_pickle('green_tripdata_2018_07.pkl')
gre_18_08.to_pickle('green_tripdata_2018_08.pkl')
yel_18_07.to_pickle('yellow_tripdata_2018_07.pkl')
yel_18_08.to_pickle('yellow_tripdata_2018_08.pkl')

In [5]:
df_pickle_yel_18_07 = pd.read_pickle('yellow_tripdata_2018_07.pkl')
df_pickle_yel_18_08 = pd.read_pickle('yellow_tripdata_2018_08.pkl')
df_pickle_yel_19_07 = pd.read_pickle('yellow_tripdata_2019_07.pkl')
df_pickle_yel_19_08 = pd.read_pickle('yellow_tripdata_2019_08.pkl')


df_pickle_gre_18_07 = pd.read_pickle('green_tripdata_2018_07.pkl')
df_pickle_gre_18_08 = pd.read_pickle('green_tripdata_2018_08.pkl')
df_pickle_gre_19_07 = pd.read_pickle('green_tripdata_2019_07.pkl')
df_pickle_gre_19_08 = pd.read_pickle('green_tripdata_2019_08.pkl')

### Preprocessing of External datasets

In [6]:
weather = pd.read_csv("central_park_weather.csv")
weather.to_pickle("weather.pkl")
df_pickle_weather = pd.read_pickle("weather.pkl")

###### The weather and traffic speed data are preprocessed in other notebooks, and loaded here. 
###### Because the dataset is too big to upload, I use Jupyter notebook to preprocess the data first.

In [7]:
df_weather = pd.read_csv("new_df_weather") #Preprocess weather dataset.ipynb
df_speed = pd.read_csv("new_df_traffic") #Preprocessing of traffic data.ipynb

###### combine the datasets together for visualisation and other analysis

In [8]:
df_yel_19 = df_pickle_yel_19_07.append([df_pickle_yel_19_08])
df_yel_18 = df_pickle_yel_18_07.append([df_pickle_yel_18_08])

df_gre_19 = df_pickle_gre_19_07.append([df_pickle_gre_19_08])
df_gre_18 = df_pickle_gre_18_07.append([df_pickle_gre_18_08])

In [9]:
df_yel = df_yel_18.append([df_yel_19])
df_gre = df_gre_18.append([df_gre_19])

In [10]:
#Filtered all instances that have 0 passenger count
df_yel = df_yel[df_yel["passenger_count"]!=0]
df_gre = df_gre[df_gre["passenger_count"]!=0]

#Filtered all instances that have 0 trip distance
df_yel = df_yel[df_yel["trip_distance"]!=0]
df_gre = df_gre[df_gre["trip_distance"]!=0]

#Filtered all instances that have payment 1 and 2. 
#I didn't consider tips as a relevent attribute in my study, so card and cash payments are all good.
df_yel = df_yel[(df_yel["payment_type"]<=2.0)]
df_gre = df_gre[(df_gre["payment_type"]<=2.0)]

#Filtered RateCodeID by only choose the Standard rate.
df_yel = df_yel[(df_yel["RatecodeID"]==1.0)]
df_gre = df_gre[(df_gre["RatecodeID"]==1.0)]

In [11]:
#format datetime
#repeat the same precedure for green taxi
df_yel['tpep_pickup_datetime']=pd.to_datetime(df_yel['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
df_yel['tpep_dropoff_datetime']=pd.to_datetime(df_yel['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')

df_gre['lpep_pickup_datetime']=pd.to_datetime(df_gre['lpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
df_gre['lpep_dropoff_datetime']=pd.to_datetime(df_gre['lpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')


#Splitting timestamp column into separate date and time columns
df_yel['Dates_pickup'] = pd.to_datetime(df_yel['tpep_pickup_datetime']).dt.date
df_yel['Time_pickup'] = pd.to_datetime(df_yel['tpep_pickup_datetime']).dt.time

df_yel['Dates_dropoff'] = pd.to_datetime(df_yel['tpep_dropoff_datetime']).dt.date
df_yel['Time_dropoff'] = pd.to_datetime(df_yel['tpep_dropoff_datetime']).dt.time

#Splitting timestamp column into separate date and time columns
df_gre['Dates_pickup'] = pd.to_datetime(df_gre['lpep_pickup_datetime']).dt.date
df_gre['Time_pickup'] = pd.to_datetime(df_gre['lpep_pickup_datetime']).dt.time

df_gre['Dates_dropoff'] = pd.to_datetime(df_gre['lpep_dropoff_datetime']).dt.date
df_gre['Time_dropoff'] = pd.to_datetime(df_gre['lpep_dropoff_datetime']).dt.time

In [12]:
df_gre.dtypes

VendorID                        float64
lpep_pickup_datetime     datetime64[ns]
lpep_dropoff_datetime    datetime64[ns]
store_and_fwd_flag               object
RatecodeID                      float64
PULocationID                      int64
DOLocationID                      int64
passenger_count                 float64
trip_distance                   float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
ehail_fee                       float64
improvement_surcharge           float64
total_amount                    float64
payment_type                    float64
trip_type                       float64
congestion_surcharge            float64
Dates_pickup                     object
Time_pickup                      object
Dates_dropoff                    object
Time_dropoff                     object
dtype: object

In [13]:
df_yel.dtypes

VendorID                        float64
tpep_pickup_datetime     datetime64[ns]
tpep_dropoff_datetime    datetime64[ns]
passenger_count                 float64
trip_distance                   float64
RatecodeID                      float64
store_and_fwd_flag               object
PULocationID                      int64
DOLocationID                      int64
payment_type                    float64
fare_amount                     float64
extra                           float64
mta_tax                         float64
tip_amount                      float64
tolls_amount                    float64
improvement_surcharge           float64
total_amount                    float64
congestion_surcharge            float64
Dates_pickup                     object
Time_pickup                      object
Dates_dropoff                    object
Time_dropoff                     object
dtype: object

In [14]:
df_yel.head(5)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Dates_pickup,Time_pickup,Dates_dropoff,Time_dropoff
1,2.0,2019-07-01 00:46:04,2019-07-01 01:05:46,1.0,4.16,1.0,N,234,25,2.0,...,0.5,0.00,0.0,0.3,20.30,2.5,2019-07-01,00:46:04,2019-07-01,01:05:46
6,1.0,2019-07-01 00:28:09,2019-07-01 00:51:00,1.0,2.40,1.0,N,142,68,1.0,...,0.5,3.35,0.0,0.3,20.15,2.5,2019-07-01,00:28:09,2019-07-01,00:51:00
7,1.0,2019-07-01 00:57:07,2019-07-01 01:11:41,1.0,3.00,1.0,N,246,141,2.0,...,0.5,0.00,0.0,0.3,16.30,2.5,2019-07-01,00:57:07,2019-07-01,01:11:41
8,4.0,2019-07-01 00:06:16,2019-07-01 00:33:14,1.0,7.89,1.0,N,50,80,1.0,...,0.5,5.96,0.0,0.3,35.76,2.5,2019-07-01,00:06:16,2019-07-01,00:33:14
9,4.0,2019-07-01 00:37:19,2019-07-01 00:52:30,1.0,4.09,1.0,N,80,97,2.0,...,0.5,0.00,0.0,0.3,16.30,0.0,2019-07-01,00:37:19,2019-07-01,00:52:30


In [15]:
df_gre.head(5)

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,Dates_pickup,Time_pickup,Dates_dropoff,Time_dropoff
0,2.0,2018-07-01 00:41:27,2018-07-01 01:06:15,N,1.0,33,50,5.0,6.37,22.5,...,NaN,0.3,28.56,1.0,1.0,NaN,2018-07-01,00:41:27,2018-07-01,01:06:15
1,1.0,2018-07-01 00:43:30,2018-07-01 00:59:51,N,1.0,25,224,1.0,5.30,18.0,...,NaN,0.3,22.56,1.0,1.0,NaN,2018-07-01,00:43:30,2018-07-01,00:59:51
2,2.0,2018-07-01 01:00:40,2018-07-01 01:13:36,N,1.0,65,225,1.0,2.98,12.0,...,NaN,0.3,13.30,2.0,1.0,NaN,2018-07-01,01:00:40,2018-07-01,01:13:36
3,1.0,2018-07-01 00:23:43,2018-07-01 00:32:49,N,1.0,7,223,1.0,1.40,8.0,...,NaN,0.3,9.30,2.0,1.0,NaN,2018-07-01,00:23:43,2018-07-01,00:32:49
4,2.0,2018-07-01 00:20:41,2018-07-01 00:24:05,N,1.0,33,25,2.0,1.03,5.0,...,NaN,0.3,8.19,1.0,1.0,NaN,2018-07-01,00:20:41,2018-07-01,00:24:05


In [16]:
#Filtered any instances that have the same pickup and dropoff time
df_yel = df_yel[df_yel["tpep_pickup_datetime"]!=df_yel["tpep_dropoff_datetime"]]
df_gre = df_gre[df_gre["lpep_pickup_datetime"]!=df_gre["lpep_dropoff_datetime"]]

In [ ]:
#Fill all NaN value with 0
df_yel = df_yel.fillna(0)
df_gre = df_gre.fillna(0)

In [ ]:
#Select only wanted study attributes
df_yel = df_yel[["PULocationID", "DOLocationID", "passenger_count", "trip_distance", "fare_amount", "extra",
                 "mta_tax", "tolls_amount", "improvement_surcharge", "total_amount", "payment_type",
                 "congestion_surcharge", "Dates_pickup", "Time_pickup", "Dates_dropoff", "Time_dropoff"
                ]]

df_gre = df_gre[["PULocationID", "DOLocationID", "passenger_count", "trip_distance", "fare_amount", "extra",
                 "mta_tax", "tolls_amount", "improvement_surcharge", "total_amount", "payment_type",
                "trip_type", "congestion_surcharge", "Dates_pickup", "Time_pickup", "Dates_dropoff", "Time_dropoff"]]

## Analysis and Geospatial Visualisation

### Preliminary analysis

In [ ]:
TILE = get_provider("STAMEN_TERRAIN_RETINA")

reset_output()
output_notebook()

In [ ]:
# sf stands for shape file
sf = gpd.read_file("MAST30034_Python/data/taxi_zones/taxi_zones.shp")
zone = pd.read_csv("taxi+_zone_lookup.csv")

# Convert the geometry shape to to latitude and longitude
# Please attribute this if you are using it
sf['geometry'] = sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")

create a `gdf` with our current dataframe and shapefile data.

From `df`, we join using `PULocationID` to match from `sf`'s `LocationID`

In [ ]:
sf_Man = sf[sf["borough"] == "Manhattan"]

In [ ]:
LocationID_Man = sf_Man["LocationID"].tolist()

In [ ]:
gdf_yel = gpd.GeoDataFrame(pd.merge(df_yel, sf, left_on='PULocationID', right_on='LocationID')).drop('PULocationID',axis=1)
gdf_gre = gpd.GeoDataFrame(pd.merge(df_gre, sf, left_on='PULocationID', right_on='LocationID')).drop('PULocationID',axis=1)

In [ ]:
geoJSON_yel = gdf_yel[['LocationID','geometry']].drop_duplicates('LocationID').to_json()
geoJSON_gre = gdf_gre[['LocationID','geometry']].drop_duplicates('LocationID').to_json()

In [ ]:
m_yel = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on how to plot aggregated data.
m_yel.add_child(folium.Choropleth(
    geo_data=geoJSON_yel,
    name='choropleth',
))

m_yel.save('MAST30034_Python/plots/foliumChoroplethMap_yel.html')
m_yel

In [ ]:
m_gre = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on how to plot aggregated data.
m_gre.add_child(folium.Choropleth(
    geo_data=geoJSON_gre,
    name='choropleth',
))

m_gre.save('MAST30034_Python/plots/foliumChoroplethMap_gre.html')
m_gre

In [ ]:
m_trip_distance = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
folium.Choropleth(
    geo_data=geoJSON_yel, # geoJSON 
    name='choropleth', # name of plot
    data=gdf_yel, # data source
    columns=['LocationID','fare_amount'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='OrRd', # color scheme
    fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Trips' # legend title
).add_to(m_trip_distance)

m_trip_distance.save('MAST30034_Python/plots/foliumChoroplethMapTrips_yel.html')
m_trip_distance

In [ ]:
#Here I want the location only in Manhattan
df_yel = df_yel[df_yel["PULocationID"].isin(LocationID_Man) & df_yel["DOLocationID"].isin(LocationID_Man)]
df_gre = df_gre[df_gre["PULocationID"].isin(LocationID_Man) & df_gre["DOLocationID"].isin(LocationID_Man)]

In [ ]:
gdf_yel = gpd.GeoDataFrame(pd.merge(df_yel, sf, left_on='PULocationID', right_on='LocationID')).drop('PULocationID',axis=1)
gdf_gre = gpd.GeoDataFrame(pd.merge(df_gre, sf, left_on='PULocationID', right_on='LocationID')).drop('PULocationID',axis=1)
geoJSON_yel = gdf_yel[['LocationID','geometry']].drop_duplicates('LocationID').to_json()
geoJSON_gre = gdf_gre[['LocationID','geometry']].drop_duplicates('LocationID').to_json()

In [ ]:
m_trip_distance = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
folium.Choropleth(
    geo_data=geoJSON_yel, # geoJSON 
    name='choropleth', # name of plot
    data=gdf_yel, # data source
    columns=['LocationID','fare_amount'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='OrRd', # color scheme
    fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Trips' # legend title
).add_to(m_trip_distance)

m_trip_distance.save('MAST30034_Python/plots/foliumChoroplethMapTrips_yel_Man.html')
m_trip_distance

In [ ]:
m_trip_distance = folium.Map(location=[40.66, -73.94], tiles="Stamen Terrain", zoom_start=10)

# refer to the folium documentations on more information on how to plot aggregated data.
folium.Choropleth(
    geo_data=geoJSON_gre, # geoJSON 
    name='choropleth', # name of plot
    data=gdf_gre, # data source
    columns=['LocationID','fare_amount'], # the columns required
    key_on='properties.LocationID', # this is from the geoJSON's properties
    fill_color='OrRd', # color scheme
    fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Trips' # legend title
).add_to(m_trip_distance)

m_trip_distance.save('MAST30034_Python/plots/foliumChoroplethMapTrips_gre_Man.html')
m_trip_distance

In [ ]:
#group by the dataset by dates
new_df_yel = df_yel.groupby([df_yel['Dates_pickup']])["passenger_count", "trip_distance", "fare_amount", 
                                                      "extra","mta_tax", "tolls_amount", "improvement_surcharge", 
                                                      "total_amount", "congestion_surcharge"].mean()
new_df_gre = df_gre.groupby([df_gre['Dates_pickup']])["passenger_count", "trip_distance", "fare_amount", 
                                                      "extra","mta_tax", "tolls_amount", "improvement_surcharge", 
                                                      "total_amount", "congestion_surcharge"].mean()
                                          

In [ ]:
weather = df_weather[""]


df_yel_all = new_df_yel
df_weather
df_speed
columns = ['Score E','Score F']

df_add = pd.DataFrame(data=data,columns=columns)

print(df)

df = pd.concat([df,df_add], axis=1)

In [ ]:
df_weather

In [ ]:
df_speed